In [18]:
#!/usr/bin/env python3
"""
Patient Timeline Creator
Creates a comprehensive timeline for patients by combining data from multiple tables.
"""

import pandas as pd
import duckdb as db
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [19]:

def get_database_connection():
    """Create and return a connection to the clinisys_all database"""
    path_to_db = '../../database/huntington_data_lake.duckdb'
    conn = db.connect(path_to_db, read_only=True)
    
    print(f"Connected to database: {path_to_db}")
    print(f"Database file exists: {os.path.exists(path_to_db)}")
    
    return conn

def extract_timeline_data_for_patient(conn, test_prontuario):
    """Extract timeline data for a specific patient"""
    query = f"""
        SELECT * FROM gold.all_patients_timeline
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_finops_data(conn, test_prontuario):
    """Get finops data for a specific patient"""
    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_mescladas(conn, test_prontuario):
    """Get mescladas data for a specific patient"""
    query = f"""
        SELECT * FROM silver.mesclada_vendas
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def main(test_prontuario):
    """Main function to create patient timeline"""
    
    # Test with a single patient first
    # test_prontuario = 175583
    
    print(f"Creating timeline for patient: {test_prontuario}")
    
    # Get database connection
    conn = get_database_connection()
    
    try:
        # Extract timeline data
        timeline_df = extract_timeline_data_for_patient(conn, test_prontuario)
       
        finops_df = get_finops_data(conn, test_prontuario)

        mescladas_df = get_mescladas(conn, test_prontuario)
        return timeline_df, finops_df, mescladas_df
        
    finally:
        conn.close()

In [30]:
if __name__ == "__main__":

    ## casos para testes iniciais

    # patient_id = 825_890 # Claudia
    # patient_id = 175_583 # Renata
    # patient_id = 220_783 # caso cabelud0:: - 26 tentativas
    # patient_id = 182_925 # caso completo
    # patient_id = 162_173 # caso bonito: 5 ciclos - 5 pagamentos

    ## resolvidos
    # patient_id = 641_950 # 11 cycles without payment
    # patient_id = 150_491 # ovo recepção
    # patient_id = 155_282 # medico externo
    # patient_id = 738_349 # (marido 738_350 não aparece no clinisys) -- sem pagamento
    # patient_id = 839574 # utero de substituicao (815299)
    # patient_id = 842663 # doadora
    # patient_id = 678_207 # numero cliente duplicado no diario_vendas (78207) - BH
    # patient_id = 831_609 # pagamento no ID errado do paciente, mas certo no marido (CLiente = 833_788)
    # patient_id = 662_298 # paciente 50347 em mescladas, cliente 62298 no diario - BH
    

    ## abertos


    ## sem pagamento
    # patient_id = 832_556 
    # patient_id = 172_179
    # patient_id = 523_966 
    # patient_id = 874_870
    # patient_id = 881_272


    patient_id = 832_556



    timeline_df, finops_df, mescladas_df = main(patient_id)


Creating timeline for patient: 832556
Connected to database: ../../database/huntington_data_lake.duckdb
Database file exists: True


In [31]:
finops_df

,prontuario,cycle_with_transfer,cycle_without_transfer,cycle_total,treatment_paid_count,treatment_paid_total,timeline_first_date,timeline_last_date,billing_first_date,billing_last_date,flag_is_donor,flag_cycles_no_payments,flag_more_cycles_than_payments,flag_no_cycles_but_payments,flag_less_cycles_than_payments,timeline_unidade
0,832556.0,1,0,1,0.0,0.0,2025-06-07,2025-06-07,NaT,NaT,0,1,1,0,0,1. HTT SP - Ibirapuera


In [32]:
timeline_df

,prontuario,event_id,event_date,reference,reference_value,tentativa,unidade,resultado_tratamento,flag_date_estimated,additional_info
0,832556.0,1444917,2025-07-15,extrato_atendimentos,1ª Consulta Nutricionista,None,4. HTT SP - Campinas,None,False,{}
1,832556.0,1439805,2025-07-11,extrato_atendimentos,1ª Consulta Nutricionista,None,4. HTT SP - Campinas,None,False,{}
2,832556.0,1428474,2025-06-27,extrato_atendimentos,Acompanhamento Beta HCG,None,4. HTT SP - Campinas,None,False,{}
3,832556.0,1428471,2025-06-20,extrato_atendimentos,Acompanhamento Beta HCG,None,4. HTT SP - Campinas,None,False,{}
4,832556.0,1423952,2025-06-17,extrato_atendimentos,Acompanhamento Beta HCG,None,4. HTT SP - Campinas,None,False,{}
5,832556.0,34830,2025-06-07,tratamentos,Ciclo de Congelados,1,4. HTT SP - Campinas,Gestação Clínica,False,{}
6,832556.0,1423949,2025-06-05,extrato_atendimentos,Resultado Exames,None,4. HTT SP - Campinas,None,False,{}
7,832556.0,1416531,2025-05-31,extrato_atendimentos,Resultado Exames,None,4. HTT SP - Campinas,None,False,{}
8,832556.0,1416532,2025-05-26,extrato_atendimentos,Resultado Exames,None,4. HTT SP - Campinas,None,False,{}
9,832556.0,1195888,2024-08-23,extrato_atendimentos,1ª Consulta Psicologia,None,1. HTT SP - Ibirapuera,None,False,{}


In [33]:
timeline_df.loc[5, 'additional_info']

'{}'

In [34]:
mescladas_df.sort_values(by='DT Emissao', ascending=False)

,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
